""" Script for cleaning up data acquired by using pphrase (https://github.com/merionum/pphrase)
    and adapting it for use with our pretrained fastText model to predict prepositional senses.
    Current preprocessing procedure is recommended for best results but can be altered if desired.
"""

In [1]:
import pandas as pd

### Preparing list of all preps found in training data

In [2]:
filename_gold = 'prep_phrases_gold.csv'
gold = pd.read_csv(filename_gold)

gold_preps = gold.prep.unique().tolist()

### Preprocessing your data

In [3]:
filename = 'oracles.csv'
df = pd.read_csv(filename, index_col = 0)

In [4]:
# leave only valid PPs:
df = df.loc[df['preposition'].isin(gold_preps)]

# replace 4-digit tokens with YEAR, any other number with NUM:
df.replace(r'\b\d{4}\b', 'YEAR', regex = True, inplace = True)
df.replace(r'\d{1,}\b', 'NUM', regex = True, inplace = True)

# bring all preps to lowercase:
df['preposition'] = df['preposition'].str.lower()

# bring preps to basic forms:
df['preposition'].replace(to_replace = {'во':'в', 'со':'с', 'ко':'к', 
                                        'об':'о', 'обо':'о', 'изо':'из', 
                                        'ото':'от', 'безо':'без', 'передо':'перед', 
                                        'надо':'над', 'подо':'под', 'из-подо':'из-под'}, 
                          inplace = True)

#remove problem characters from host tokens:
df['host_lemma'] = df['host_lemma'].replace(r"[\"\'\(\)«»“”\!\?\,\.\:]", '', regex = True)
#bring host tokens to lowercase:
df['host_lemma'] = df['host_lemma'].str.lower()

#remove problem characters from dependent tokens:
df['dependant_lemma'] = df['dependant_lemma'].replace(r"[\"\'\(\)«»“”\!\?\,\.\:]", "", regex = True)
#bring dependent tokens to lowercase:
df['dependant_lemma'] = df['dependant_lemma'].str.lower()

#remove incomplete PPs
df.dropna(axis = 0, inplace = True)
df.reset_index(drop = True, inplace = True)

In [5]:
# tag parsing + dependent case isolation 
df_tags = df['dependant_tags'].copy()

for i in df_tags.index:
    tagset = df_tags.at[i].split('|')
    tagset = [x for x in tagset if '=' in x]
    df_tags.at[i] = {x.split('=')[0]: x.split('=')[1].strip() for x in tagset}
    
df = df.join(pd.DataFrame(list(df_tags)).Case)

In [6]:
df

,phrase,host,preposition,dependant,full_dependant,host_tags,dependant_tags,host_lemma,dependant_lemma,Case
0,подходят в отдельных ситуациях,подходят,в,ситуациях,отдельных ситуациях,PartOfSpeech=VERB|Aspect=Imp|Mood=Ind|Number=P...,PartOfSpeech=NOUN|Animacy=Inan|Case=Loc|Gender...,подходить,ситуация,Loc
1,указывают на проблему,указывают,на,проблему,проблему,PartOfSpeech=VERB|Aspect=Imp|Mood=Ind|Number=P...,PartOfSpeech=NOUN|Animacy=Inan|Case=Acc|Gender...,указывать,проблема,Acc
2,продукта с предыдущей,продукта,с,предыдущей,предыдущей,PartOfSpeech=NOUN|Animacy=Inan|Case=Gen|Gender...,PartOfSpeech=ADJ|Case=Gen|Degree=Pos|Gender=Fe...,продукт,предыдущий,Gen
3,о ней говорят,говорят,о,ней,ней,PartOfSpeech=VERB|Aspect=Imp|Mood=Ind|Number=P...,PartOfSpeech=PRON|Case=Loc|Gender=Fem|Number=S...,говорить,она,Loc
4,"выводы с встреч , планёрок , долгосрочного пла...",выводы,с,встреч,"встреч , планёрок , долгосрочного планирования",PartOfSpeech=NOUN|Animacy=Inan|Case=Nom|Gender...,PartOfSpeech=NOUN|Animacy=Inan|Case=Gen|Gender...,вывод,встреча,Gen
5,"Убедись в том , что любой шаг работы системы м...",Убедись,в,том,"том , что любой шаг работы системы можно объяс...",PartOfSpeech=VERB|Aspect=Perf|Mood=Imp|Number=...,PartOfSpeech=PRON|Animacy=Inan|Case=Loc|Gender...,убестись,то,Loc
6,в продукте нет,нет,в,продукте,продукте,PartOfSpeech=VERB|Aspect=Imp|Mood=Ind|Number=S...,PartOfSpeech=NOUN|Animacy=Inan|Case=Loc|Gender...,нет,продукт,Loc
7,"Убедись , что в продукте есть ценности , котор...",Убедись,в,продукте,", что продукте есть ценности , которые ассоции...",PartOfSpeech=VERB|Aspect=Perf|Mood=Imp|Number=...,PartOfSpeech=NOUN|Animacy=Inan|Case=Loc|Gender...,убестись,продукт,Loc
8,ассоциируются с брендом,ассоциируются,с,брендом,брендом,PartOfSpeech=VERB|Aspect=Imp|Mood=Ind|Number=P...,PartOfSpeech=NOUN|Animacy=Inan|Case=Ins|Gender...,ассоциироваться,бренд,Ins
9,продаёт из неперерабатываемого пластика,продаёт,из,пластика,неперерабатываемого пластика,PartOfSpeech=VERB|Aspect=Imp|Mood=Ind|Number=S...,PartOfSpeech=NOUN|Animacy=Inan|Case=Gen|Gender...,продаёть,пластик,Gen


In [7]:
# 'prep_Case' format column format necessary for fastText prediction:
df.insert(len(df.columns), 'ft_prep', None)

for i in df.index:
    if pd.isna(df.at[i, 'Case']) == False:
        df.at[i,'ft_prep'] = '_'.join([df.at[i, 'preposition'], df.at[i, 'Case']])
    else:
        df.at[i,'ft_prep'] = df.at[i, 'preposition'] # if no Case found then just 'prep'

# 'host prep_Case dep' format column for fastText prediction: 
df['ft_phrase'] = df['host_lemma'].str.cat(df[['ft_prep', 'dependant_lemma']].values, sep = ' ')

In [8]:
df.to_csv(f'{filename[:-4]}_preprocessed.csv')

### Prepositional sense prediction using our model

In [9]:
import fasttext

In [10]:
filename_model = 'gold_hpd_lemma_case_ud_more.bin'
model = fasttext.load_model(filename_model)

In [11]:
for i in df.index:
    prediction = model.predict(df.at[i, 'ft_phrase'])
    df.at[i, 'label'] = prediction[0][0].strip('__label__')
    df.at[i, 'probability'] = prediction[1][0]

In [13]:
df.to_csv(f'{filename[:-4]}_predicted.csv', index = False)